In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
shift_hrs = 1
shift_steps = shift_hrs*2


data = pd.read_csv('../Data/solar_original.csv')
data['TimeStamp']=pd.to_datetime(data.Date+' '+data.Time,infer_datetime_format=True)
data['DayOfYear']=pd.to_datetime(data.Date,dayfirst=True).dt.dayofyear
data=data.drop(['Date','Time','TimeStamp'],axis=1)

Y=data['solar_output'].shift(-shift_steps)
X=data.drop(['solar_output'],axis=1)

if shift_steps == 0:
    x_data = X.values
    y_data = Y.values
else:
    x_data = X.values[0:-shift_steps]
    y_data = Y.values[0:-shift_steps]

In [3]:
data.head()

,solar_output,temperature,humidity,wind_speed,visibility,serial,DayOfYear
0,0.00000,27.0,74.0,0.0,4.0,1,275
1,0.14216,27.0,74.0,1.9,4.0,2,275
2,1.34680,27.0,74.0,0.0,4.0,3,275
3,3.37716,28.0,70.0,7.4,4.0,4,275
4,5.67044,28.0,70.0,9.3,4.0,5,275


In [76]:
Y=data['solar_output'].shift(-shift_steps)
X=data.drop(['solar_output'],axis=1)

if shift_steps == 0:
    x_data = X.values
    y_data = Y.values
else:
    x_data = X.values[0:-shift_steps]
    y_data = Y.values[0:-shift_steps]
    

In [4]:
x_data, y_data

(array([[ 27.  ,  74.  ,   0.  ,   4.  ,   1.  , 275.  ],
        [ 27.  ,  74.  ,   1.9 ,   4.  ,   2.  , 275.  ],
        [ 27.  ,  74.  ,   0.  ,   4.  ,   3.  , 275.  ],
        ...,
        [ 40.  ,   6.  ,  13.  ,   6.  ,  21.  ,  89.  ],
        [ 40.  ,   6.  ,  11.79,   6.  ,  22.  ,  89.  ],
        [ 40.  ,   6.  ,  11.1 ,   6.  ,  23.  ,  89.  ]]),
 array([1.3468 , 3.37716, 5.67044, ..., 7.60269, 4.1613 , 1.59809]))

In [5]:
print (y_data.shape,x_data.shape)

(4498,) (4498, 6)


In [6]:
## Train Test Split

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

x_scaler = MinMaxScaler()
x_data_scaled=x_scaler.fit_transform(x_data)

y_scaler = MinMaxScaler()
y_data_scaled=y_scaler.fit_transform(y_data.reshape(-1, 1)).ravel()

# Changes done Here !!! 
x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(x_data_scaled, y_data, test_size=0.2,random_state=40)

In [7]:
## [rows, timesteps, columns]
x_train_scaled=x_train_scaled.reshape(x_train_scaled.shape[0],1,x_train_scaled.shape[1])
x_test_scaled=x_test_scaled.reshape(x_test_scaled.shape[0],1,x_test_scaled.shape[1])

# print(scaler.inverse_transform([[ 1.5  0. ]]))

In [16]:
# design network
n_batch=1
model = keras.models.Sequential()
model.add(keras.layers.LSTM(10, batch_input_shape=(n_batch, x_train_scaled.shape[1], x_train_scaled.shape[2]), 
                            return_sequences=True))
model.add(keras.layers.LSTM(10,return_sequences=True))
model.add(keras.layers.LSTM(10,return_sequences=True))
model.add(keras.layers.LSTM(10,return_sequences=True))
model.add(keras.layers.LSTM(10,stateful=True))
model.add(keras.layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae'])


In [17]:
# fit network
n_epoch=300
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
for i in range(n_epoch):
    model.fit(x_train_scaled, y_train_scaled, epochs=1,validation_split=0.2,callbacks=[early_stop], batch_size=n_batch, verbose=1, shuffle=False)
    model.reset_states()
# batch forecast


Train on 2878 samples, validate on 720 samples
Epoch 1/1
2878/2878 [==============================] - 54s 19ms/step - loss: 84.2247 - mean_absolute_error: 8.1378 - val_loss: 76.3259 - val_mean_absolute_error: 7.9400
Train on 2878 samples, validate on 720 samples
Epoch 1/1
 321/2878 [==>...........................] - ETA: 42s - loss: 73.2756 - mean_absolute_error: 7.7210

KeyboardInterrupt: 

In [ ]:
y_predicted_scaled=model.predict(x_test_scaled, batch_size=n_batch)
y_predicted_notscaled=y_scaler.inverse_transform(y_predicted_scaled)

In [ ]:
from sklearn.metrics import mean_absolute_error
#mean_absolute_error(y_predicted_notscaled,y_scaler.inverse_transform(y_test_scaled.reshape(-1, 1)).ravel())
mean_absolute_error(y_predicted_scaled,y_test_scaled)

In [1]:

#[loss, mae] = model.evaluate(x_test_scaled, y_test_scaled, verbose=0)

In [343]:
## With an Hour Delay ( 10 lstm size) 18 Second for each epoch
# The Scaled Output prediction was : 0.07338819905623634
# Without Scaled 1.8047074860113639
# With scaled and using inverse transform 1.98150161730334

# Worst for Current: 2.210785236899325

## 2 layes of LSTM ( 10 LSTM Size) 27s for each Epoch
# With scaled and using inverse transform 1.756790105364927

## 3 layes of LSTM ( 10 LSTM Size) 36s for each Epoch
#  With scaled and using inverse transform 1.9142602957533301 (1.1448)
for i in range(len(y_test_scaled)):
    print('>Expected=%.1f, Predicted=%.1f' % (y_test_scaled[i], y[i]))

>Expected=0.6, Predicted=0.5
>Expected=0.4, Predicted=0.5
>Expected=0.0, Predicted=0.1
>Expected=0.5, Predicted=0.5
>Expected=0.8, Predicted=0.8
>Expected=0.6, Predicted=0.4
>Expected=0.8, Predicted=0.8
>Expected=0.0, Predicted=-0.0
>Expected=0.1, Predicted=0.1
>Expected=0.3, Predicted=0.3
>Expected=0.4, Predicted=0.5
>Expected=0.3, Predicted=0.3
>Expected=0.1, Predicted=0.2
>Expected=0.7, Predicted=0.7
>Expected=0.2, Predicted=0.3
>Expected=0.7, Predicted=0.8
>Expected=0.7, Predicted=0.9
>Expected=0.0, Predicted=0.1
>Expected=0.6, Predicted=0.7
>Expected=0.8, Predicted=0.8
>Expected=0.5, Predicted=0.4
>Expected=0.8, Predicted=0.7
>Expected=0.4, Predicted=0.5
>Expected=0.6, Predicted=0.6
>Expected=0.0, Predicted=-0.0
>Expected=0.4, Predicted=0.3
>Expected=0.2, Predicted=0.2
>Expected=0.7, Predicted=0.7
>Expected=0.6, Predicted=0.5
>Expected=0.7, Predicted=0.7
>Expected=0.7, Predicted=0.7
>Expected=0.7, Predicted=0.8
>Expected=0.9, Predicted=0.9
>Expected=0.2, Predicted=0.2
>Expected=0.